**-SNAP benefits per CB - normalized by population (% of total population or beneficiaries per 1000 residents)**

** https://data.cityofnewyork.us/Social-Services/Borough-Community-District-Report-SNAP-Population/jye8-w4d7 **


** INDEX - Borough ID __ (any separator) Community Board Number **



In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import urllib
import sys
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# SNAP data
# Data Last Updated October 27, 2020
# Metadata Last Updated October 27, 2020
# Date Created October 30, 2018
data = pd.read_csv('Borough_Community_District_Report_-_SNAP_Population.csv')
data.head()

,Month,Boro,CD,BC_SNAP_Recipients,BC_SNAP_Households
0,09/01/2018,MANHATTAN,M01,2245,1762
1,09/01/2018,MANHATTAN,M02,3170,2481
2,09/01/2018,MANHATTAN,M03,37948,25250
3,09/01/2018,MANHATTAN,M04,11664,8750
4,09/01/2018,MANHATTAN,M05,4236,3255


In [3]:
data.dtypes

Month                 object
Boro                  object
CD                    object
BC_SNAP_Recipients     int64
BC_SNAP_Households     int64
dtype: object

In [4]:
data.shape

(531, 5)

In [5]:
pd.to_datetime(data['Month'])

0     2018-09-01
1     2018-09-01
2     2018-09-01
3     2018-09-01
4     2018-09-01
         ...    
526   2020-09-01
527   2020-09-01
528   2020-09-01
529   2020-09-01
530   2020-09-01
Name: Month, Length: 531, dtype: datetime64[ns]

In [6]:
data['Month'] = pd.to_datetime(data['Month'])

In [7]:
data['CD'] = data['CD'].astype(str)

In [8]:
data.Boro.unique()

array(['MANHATTAN', 'BROOKLYN', 'BRONX', 'QUEENS', 'STATEN ISLAND',
       'Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'],
      dtype=object)

In [9]:
data['Boro'] = data.Boro.str.title()

In [10]:
# Community districts:
# Manhattan has 12 M01...
# Brooklyn has 18 K01...
# Bronx has 12 B01...
# Queens has 12 Q01...
# Staten Island has 3 S01...
data.CD.unique()

array(['M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09',
       'M10', 'M11', 'M12', 'K01', 'K02', 'K03', 'K04', 'K05', 'K06',
       'K07', 'K08', 'K09', 'K10', 'K11', 'K12', 'K13', 'K14', 'K15',
       'K16', 'K17', 'K18', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06',
       'B07', 'B08', 'B09', 'B10', 'B11', 'B12', 'Q01', 'Q02', 'Q03',
       'Q04', 'Q05', 'Q06', 'Q07', 'Q08', 'Q09', 'Q10', 'Q11', 'Q12',
       'Q13', 'Q14', 'S01', 'S02', 'S03'], dtype=object)

In [11]:
data['CD'] = data["CD"].str.replace('M', 'MN ')
data['CD'] = data["CD"].str.replace('B', 'BX ') # order matters.. need to check for better solution
data['CD'] = data["CD"].str.replace('K', 'BK ')
data['CD'] = data["CD"].str.replace('Q', 'QN ')
data['CD'] = data["CD"].str.replace('S', 'SI ')
data

,Month,Boro,CD,BC_SNAP_Recipients,BC_SNAP_Households
0,2018-09-01,Manhattan,MN 01,2245,1762
1,2018-09-01,Manhattan,MN 02,3170,2481
2,2018-09-01,Manhattan,MN 03,37948,25250
3,2018-09-01,Manhattan,MN 04,11664,8750
4,2018-09-01,Manhattan,MN 05,4236,3255
...,...,...,...,...,...
526,2020-09-01,Queens,QN 13,22301,14059
527,2020-09-01,Queens,QN 14,30889,16829
528,2020-09-01,Staten Island,SI 01,39400,20918
529,2020-09-01,Staten Island,SI 02,15810,9214


In [12]:
data.CD.unique()

array(['MN 01', 'MN 02', 'MN 03', 'MN 04', 'MN 05', 'MN 06', 'MN 07',
       'MN 08', 'MN 09', 'MN 10', 'MN 11', 'MN 12', 'BK 01', 'BK 02',
       'BK 03', 'BK 04', 'BK 05', 'BK 06', 'BK 07', 'BK 08', 'BK 09',
       'BK 10', 'BK 11', 'BK 12', 'BK 13', 'BK 14', 'BK 15', 'BK 16',
       'BK 17', 'BK 18', 'BX 01', 'BX 02', 'BX 03', 'BX 04', 'BX 05',
       'BX 06', 'BX 07', 'BX 08', 'BX 09', 'BX 10', 'BX 11', 'BX 12',
       'QN 01', 'QN 02', 'QN 03', 'QN 04', 'QN 05', 'QN 06', 'QN 07',
       'QN 08', 'QN 09', 'QN 10', 'QN 11', 'QN 12', 'QN 13', 'QN 14',
       'SI 01', 'SI 02', 'SI 03'], dtype=object)

In [13]:
Borough_ID = {
    'manhattan': 1,
    'bronx': 2,
    'brooklyn': 3,
    'queens': 4,
    'staten island': 5
}

data['Borough_ID'] = data['Boro'].str.lower().map(Borough_ID)

In [14]:
data

,Month,Boro,CD,BC_SNAP_Recipients,BC_SNAP_Households,Borough_ID
0,2018-09-01,Manhattan,MN 01,2245,1762,1
1,2018-09-01,Manhattan,MN 02,3170,2481,1
2,2018-09-01,Manhattan,MN 03,37948,25250,1
3,2018-09-01,Manhattan,MN 04,11664,8750,1
4,2018-09-01,Manhattan,MN 05,4236,3255,1
...,...,...,...,...,...,...
526,2020-09-01,Queens,QN 13,22301,14059,4
527,2020-09-01,Queens,QN 14,30889,16829,4
528,2020-09-01,Staten Island,SI 01,39400,20918,5
529,2020-09-01,Staten Island,SI 02,15810,9214,5


In [15]:
data['index'] = data[['Borough_ID', 'CD']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [16]:
data

,Month,Boro,CD,BC_SNAP_Recipients,BC_SNAP_Households,Borough_ID,index
0,2018-09-01,Manhattan,MN 01,2245,1762,1,1_MN 01
1,2018-09-01,Manhattan,MN 02,3170,2481,1,1_MN 02
2,2018-09-01,Manhattan,MN 03,37948,25250,1,1_MN 03
3,2018-09-01,Manhattan,MN 04,11664,8750,1,1_MN 04
4,2018-09-01,Manhattan,MN 05,4236,3255,1,1_MN 05
...,...,...,...,...,...,...,...
526,2020-09-01,Queens,QN 13,22301,14059,4,4_QN 13
527,2020-09-01,Queens,QN 14,30889,16829,4,4_QN 14
528,2020-09-01,Staten Island,SI 01,39400,20918,5,5_SI 01
529,2020-09-01,Staten Island,SI 02,15810,9214,5,5_SI 02


In [17]:
# SNAP benefits per CB - normalized by population (% of total population or beneficiaries per 1000 residents)

In [18]:
data2018 = data[data.Month.dt.year == 2018] # 118 rows
data2019 = data[data.Month.dt.year == 2019] # 236 rows
data2020 = data[data.Month.dt.year == 2020] # 177 rows

In [19]:
data2018

,Month,Boro,CD,BC_SNAP_Recipients,BC_SNAP_Households,Borough_ID,index
0,2018-09-01,Manhattan,MN 01,2245,1762,1,1_MN 01
1,2018-09-01,Manhattan,MN 02,3170,2481,1,1_MN 02
2,2018-09-01,Manhattan,MN 03,37948,25250,1,1_MN 03
3,2018-09-01,Manhattan,MN 04,11664,8750,1,1_MN 04
4,2018-09-01,Manhattan,MN 05,4236,3255,1,1_MN 05
...,...,...,...,...,...,...,...
113,2018-12-01,Queens,QN 13,18536,11837,4,4_QN 13
114,2018-12-01,Queens,QN 14,28537,15315,4,4_QN 14
115,2018-12-01,Staten Island,SI 01,36386,19289,5,5_SI 01
116,2018-12-01,Staten Island,SI 02,13799,8226,5,5_SI 02


In [20]:
data2019

,Month,Boro,CD,BC_SNAP_Recipients,BC_SNAP_Households,Borough_ID,index
118,2019-03-01,Bronx,BX 01,43127,23086,2,2_BX 01
119,2019-03-01,Bronx,BX 02,21203,11531,2,2_BX 02
120,2019-03-01,Bronx,BX 03,40628,21829,2,2_BX 03
121,2019-03-01,Bronx,BX 04,55067,30205,2,2_BX 04
122,2019-03-01,Bronx,BX 05,53602,29021,2,2_BX 05
...,...,...,...,...,...,...,...
349,2019-12-01,Queens,QN 13,17978,11703,4,4_QN 13
350,2019-12-01,Queens,QN 14,27394,14948,4,4_QN 14
351,2019-12-01,Staten Island,SI 01,34626,18479,5,5_SI 01
352,2019-12-01,Staten Island,SI 02,13737,8189,5,5_SI 02


In [21]:
data2020

,Month,Boro,CD,BC_SNAP_Recipients,BC_SNAP_Households,Borough_ID,index
354,2020-03-01,Bronx,BX 01,41545,22736,2,2_BX 01
355,2020-03-01,Bronx,BX 02,20279,11294,2,2_BX 02
356,2020-03-01,Bronx,BX 03,39250,21694,2,2_BX 03
357,2020-03-01,Bronx,BX 04,50360,28414,2,2_BX 04
358,2020-03-01,Bronx,BX 05,49525,27397,2,2_BX 05
...,...,...,...,...,...,...,...
526,2020-09-01,Queens,QN 13,22301,14059,4,4_QN 13
527,2020-09-01,Queens,QN 14,30889,16829,4,4_QN 14
528,2020-09-01,Staten Island,SI 01,39400,20918,5,5_SI 01
529,2020-09-01,Staten Island,SI 02,15810,9214,5,5_SI 02


In [22]:
communitypop = pd.read_csv('New_York_City_Population_By_Community_Districts.csv')
communitypop

,Borough,CD Number,CD Name,1970 Population,1980 Population,1990 Population,2000 Population,2010 Population
0,Bronx,1,"Melrose, Mott Haven, Port Morris",138557,78441,77214,82159,91497
1,Bronx,2,"Hunts Point, Longwood",99493,34399,39443,46824,52246
2,Bronx,3,"Morrisania, Crotona Park East",150636,53635,57162,68574,79762
3,Bronx,4,"Highbridge, Concourse Village",144207,114312,119962,139563,146441
4,Bronx,5,"University Hts., Fordham, Mt. Hope",121807,107995,118435,128313,128200
5,Bronx,6,"East Tremont, Belmont",114137,65016,68061,75688,83268
6,Bronx,7,"Bedford Park, Norwood, Fordham",113764,116827,128588,141411,139286
7,Bronx,8,"Riverdale, Kingsbridge, Marble Hill",103543,98275,97030,101332,101731
8,Bronx,9,"Soundview, Parkchester",166442,167627,155970,167859,172298
9,Bronx,10,"Throgs Nk., Co-op City, Pelham Bay",84948,106516,108093,115948,120392


In [23]:
communitypop = communitypop[['Borough', 'CD Number', '2010 Population']]

In [24]:
communitypop

,Borough,CD Number,2010 Population
0,Bronx,1,91497
1,Bronx,2,52246
2,Bronx,3,79762
3,Bronx,4,146441
4,Bronx,5,128200
5,Bronx,6,83268
6,Bronx,7,139286
7,Bronx,8,101731
8,Bronx,9,172298
9,Bronx,10,120392


In [25]:
data2018[['Boro', 'CD', 'Borough_ID', 'index', 'BC_SNAP_Recipients', 'BC_SNAP_Households']] \
                .groupby(by=['Boro', 'CD', 'Borough_ID', 'index']).sum()

BC_SNAP_Recipients  BC_SNAP_Households
Boro          CD    Borough_ID index                                          
Bronx         BX 01 2          2_BX 01               89216               47287
              BX 02 2          2_BX 02               44528               24013
              BX 03 2          2_BX 03               82884               44193
              BX 04 2          2_BX 04              115204               62630
              BX 05 2          2_BX 05              112023               60231
              BX 06 2          2_BX 06               77925               41358
              BX 07 2          2_BX 07               92794               51528
              BX 08 2          2_BX 08               32575               19374
              BX 09 2          2_BX 09              105560               57194
              BX 10 2          2_BX 10               35048               21181
              BX 11 2          2_BX 11               47557               27411
              BX 12 2          2_BX 12               74121               40290
Brooklyn      BK 01 3          3_BK 01              103326               40463
              BK 02 3          3_BK 02               26896               17876
              BK 03 3          3_BK 03               89022               47418
              BK 04 3          3_BK 04               50338               30011
              BK 05 3          3_BK 05              125396               69984
              BK 06 3          3_BK 06               19725               12883
              BK 07 3          3_BK 07               44732               25656
              BK 08 3          3_BK 08               41556               26121
              BK 09 3          3_BK 09               41573               24792
              BK 10 3          3_BK 10               33972               19454
              BK 11 3          3_BK 11               73315               42452
              BK 12 3          3_BK 12              123171               40186
              BK 13 3          3_BK 13               64530               42098
              BK 14 3          3_BK 14               63811               36670
              BK 15 3          3_BK 15               56453               34412
              BK 16 3          3_BK 16               73037               40208
              BK 17 3          3_BK 17               60776               37597
              BK 18 3          3_BK 18               55536               32959
Manhattan     MN 01 1          1_MN 01                4336                3442
              MN 02 1          1_MN 02                6107                4814
              MN 03 1          1_MN 03               74016               49761
              MN 04 1          1_MN 04               23193               17421
              MN 05 1          1_MN 05                9943                7084
              MN 06 1          1_MN 06                9892                8514
              MN 07 1          1_MN 07               32566               24422
              MN 08 1          1_MN 08               11116                8814
              MN 09 1          1_MN 09               44881               29082
              MN 10 1          1_MN 10               67169               42157
              MN 11 1          1_MN 11               84957               52479
              MN 12 1          1_MN 12               94729               63683
Queens        QN 01 4          4_QN 01               44724               26567
              QN 02 4          4_QN 02               21531               13385
              QN 03 4          4_QN 03               50776               28852
              QN 04 4          4_QN 04               52785               30250
              QN 05 4          4_QN 05               36659               21106
              QN 06 4          4_QN 06               20094               13868
              QN 07 4          4_QN 07               51866               33857
           

In [26]:
group2018 = data2018[['Boro', 'CD', 'Borough_ID', 'index', 'BC_SNAP_Recipients', 'BC_SNAP_Households']] \
                .groupby(by=['Boro', 'CD', 'Borough_ID', 'index'], as_index=False).sum()

In [27]:
len(group2018)

59

In [28]:
len(communitypop)

59

In [29]:
group2018['total_pop'] = communitypop['2010 Population'].values

In [30]:
group2018

,Boro,CD,Borough_ID,index,BC_SNAP_Recipients,BC_SNAP_Households,total_pop
0,Bronx,BX 01,2,2_BX 01,89216,47287,91497
1,Bronx,BX 02,2,2_BX 02,44528,24013,52246
2,Bronx,BX 03,2,2_BX 03,82884,44193,79762
3,Bronx,BX 04,2,2_BX 04,115204,62630,146441
4,Bronx,BX 05,2,2_BX 05,112023,60231,128200
5,Bronx,BX 06,2,2_BX 06,77925,41358,83268
6,Bronx,BX 07,2,2_BX 07,92794,51528,139286
7,Bronx,BX 08,2,2_BX 08,32575,19374,101731
8,Bronx,BX 09,2,2_BX 09,105560,57194,172298
9,Bronx,BX 10,2,2_BX 10,35048,21181,120392


In [31]:
# benefi_by_Boro_Community_per_1000
group2018['yearly_ind_benefi_per_1000']=(group2018['BC_SNAP_Recipients']/group2018['total_pop'])*1000
group2018['yearly_household_benefi_per_1000']=(group2018['BC_SNAP_Households']/group2018['total_pop'])*1000

In [32]:
group2018

,Boro,CD,Borough_ID,index,BC_SNAP_Recipients,BC_SNAP_Households,total_pop,yearly_ind_benefi_per_1000,yearly_household_benefi_per_1000
0,Bronx,BX 01,2,2_BX 01,89216,47287,91497,975.070221,516.814759
1,Bronx,BX 02,2,2_BX 02,44528,24013,52246,852.275772,459.614133
2,Bronx,BX 03,2,2_BX 03,82884,44193,79762,1039.141446,554.060831
3,Bronx,BX 04,2,2_BX 04,115204,62630,146441,786.692251,427.680772
4,Bronx,BX 05,2,2_BX 05,112023,60231,128200,873.814353,469.820593
5,Bronx,BX 06,2,2_BX 06,77925,41358,83268,935.833694,496.685401
6,Bronx,BX 07,2,2_BX 07,92794,51528,139286,666.211967,369.943857
7,Bronx,BX 08,2,2_BX 08,32575,19374,101731,320.207213,190.443424
8,Bronx,BX 09,2,2_BX 09,105560,57194,172298,612.659462,331.948136
9,Bronx,BX 10,2,2_BX 10,35048,21181,120392,291.115689,175.933617


In [33]:
group2019 = data2019[['Boro', 'CD', 'Borough_ID', 'index', 'BC_SNAP_Recipients', 'BC_SNAP_Households']] \
                .groupby(by=['Boro', 'CD', 'Borough_ID', 'index'], as_index=False).sum()

In [34]:
group2019['total_pop'] = communitypop['2010 Population'].values

In [35]:
group2019

,Boro,CD,Borough_ID,index,BC_SNAP_Recipients,BC_SNAP_Households,total_pop
0,Bronx,BX 01,2,2_BX 01,169162,91364,91497
1,Bronx,BX 02,2,2_BX 02,82825,45534,52246
2,Bronx,BX 03,2,2_BX 03,158966,86488,79762
3,Bronx,BX 04,2,2_BX 04,211942,117646,146441
4,Bronx,BX 05,2,2_BX 05,205934,112569,128200
5,Bronx,BX 06,2,2_BX 06,148748,80523,83268
6,Bronx,BX 07,2,2_BX 07,172237,98522,139286
7,Bronx,BX 08,2,2_BX 08,61402,36905,101731
8,Bronx,BX 09,2,2_BX 09,198497,109490,172298
9,Bronx,BX 10,2,2_BX 10,67807,41607,120392


In [36]:
# benefi_by_Boro_Community_per_1000
group2019['yearly_ind_benefi_per_1000']=(group2019['BC_SNAP_Recipients']/group2019['total_pop'])*1000
group2019['yearly_household_benefi_per_1000']=(group2019['BC_SNAP_Households']/group2019['total_pop'])*1000

In [37]:
group2019

,Boro,CD,Borough_ID,index,BC_SNAP_Recipients,BC_SNAP_Households,total_pop,yearly_ind_benefi_per_1000,yearly_household_benefi_per_1000
0,Bronx,BX 01,2,2_BX 01,169162,91364,91497,1848.825645,998.546400
1,Bronx,BX 02,2,2_BX 02,82825,45534,52246,1585.288826,871.530835
2,Bronx,BX 03,2,2_BX 03,158966,86488,79762,1993.004187,1084.325869
3,Bronx,BX 04,2,2_BX 04,211942,117646,146441,1447.285938,803.367909
4,Bronx,BX 05,2,2_BX 05,205934,112569,128200,1606.349454,878.073323
5,Bronx,BX 06,2,2_BX 06,148748,80523,83268,1786.376519,967.034155
6,Bronx,BX 07,2,2_BX 07,172237,98522,139286,1236.570797,707.335985
7,Bronx,BX 08,2,2_BX 08,61402,36905,101731,603.572166,362.770444
8,Bronx,BX 09,2,2_BX 09,198497,109490,172298,1152.056321,635.468781
9,Bronx,BX 10,2,2_BX 10,67807,41607,120392,563.218486,345.596053


In [38]:
group2020 = data2020[['Boro', 'CD', 'Borough_ID', 'index', 'BC_SNAP_Recipients', 'BC_SNAP_Households']] \
                .groupby(by=['Boro', 'CD', 'Borough_ID', 'index'], as_index=False).sum()

In [39]:
group2020['total_pop'] = communitypop['2010 Population'].values

In [40]:
group2020

,Boro,CD,Borough_ID,index,BC_SNAP_Recipients,BC_SNAP_Households,total_pop
0,Bronx,BX 01,2,2_BX 01,134394,73308,91497
1,Bronx,BX 02,2,2_BX 02,65849,36575,52246
2,Bronx,BX 03,2,2_BX 03,127653,70233,79762
3,Bronx,BX 04,2,2_BX 04,164763,92475,146441
4,Bronx,BX 05,2,2_BX 05,161304,89080,128200
5,Bronx,BX 06,2,2_BX 06,117260,64187,83268
6,Bronx,BX 07,2,2_BX 07,137183,79007,139286
7,Bronx,BX 08,2,2_BX 08,51105,30667,101731
8,Bronx,BX 09,2,2_BX 09,159601,88372,172298
9,Bronx,BX 10,2,2_BX 10,56213,34297,120392


In [41]:
# benefi_by_Boro_Community_per_1000
group2020['yearly_ind_benefi_per_1000']=(group2020['BC_SNAP_Recipients']/group2020['total_pop'])*1000
group2020['yearly_household_benefi_per_1000']=(group2020['BC_SNAP_Households']/group2020['total_pop'])*1000

In [42]:
group2020

,Boro,CD,Borough_ID,index,BC_SNAP_Recipients,BC_SNAP_Households,total_pop,yearly_ind_benefi_per_1000,yearly_household_benefi_per_1000
0,Bronx,BX 01,2,2_BX 01,134394,73308,91497,1468.835044,801.206597
1,Bronx,BX 02,2,2_BX 02,65849,36575,52246,1260.364430,700.053593
2,Bronx,BX 03,2,2_BX 03,127653,70233,79762,1600.423761,880.532083
3,Bronx,BX 04,2,2_BX 04,164763,92475,146441,1125.115234,631.482986
4,Bronx,BX 05,2,2_BX 05,161304,89080,128200,1258.221529,694.851794
5,Bronx,BX 06,2,2_BX 06,117260,64187,83268,1408.224048,770.848345
6,Bronx,BX 07,2,2_BX 07,137183,79007,139286,984.901569,567.228580
7,Bronx,BX 08,2,2_BX 08,51105,30667,101731,502.354248,301.451868
8,Bronx,BX 09,2,2_BX 09,159601,88372,172298,926.307908,512.902065
9,Bronx,BX 10,2,2_BX 10,56213,34297,120392,466.916406,284.877733


In [43]:
# df2018 =
group2018[['index', 'yearly_ind_benefi_per_1000', 'yearly_household_benefi_per_1000']].set_index(['index'])

,yearly_ind_benefi_per_1000,yearly_household_benefi_per_1000
index,,
2_BX 01,975.070221,516.814759
2_BX 02,852.275772,459.614133
2_BX 03,1039.141446,554.060831
2_BX 04,786.692251,427.680772
2_BX 05,873.814353,469.820593
2_BX 06,935.833694,496.685401
2_BX 07,666.211967,369.943857
2_BX 08,320.207213,190.443424
2_BX 09,612.659462,331.948136


In [44]:
df2018 = group2018[['index', 'yearly_ind_benefi_per_1000', 'yearly_household_benefi_per_1000']].set_index(['index'])
df2019 = group2019[['index', 'yearly_ind_benefi_per_1000', 'yearly_household_benefi_per_1000']].set_index(['index'])
df2020 = group2020[['index', 'yearly_ind_benefi_per_1000', 'yearly_household_benefi_per_1000']].set_index(['index'])

In [45]:
df2018.head()

,yearly_ind_benefi_per_1000,yearly_household_benefi_per_1000
index,,
2_BX 01,975.070221,516.814759
2_BX 02,852.275772,459.614133
2_BX 03,1039.141446,554.060831
2_BX 04,786.692251,427.680772
2_BX 05,873.814353,469.820593


In [46]:
df2019.head()

,yearly_ind_benefi_per_1000,yearly_household_benefi_per_1000
index,,
2_BX 01,1848.825645,998.546400
2_BX 02,1585.288826,871.530835
2_BX 03,1993.004187,1084.325869
2_BX 04,1447.285938,803.367909
2_BX 05,1606.349454,878.073323


In [47]:
df2020.head()

,yearly_ind_benefi_per_1000,yearly_household_benefi_per_1000
index,,
2_BX 01,1468.835044,801.206597
2_BX 02,1260.364430,700.053593
2_BX 03,1600.423761,880.532083
2_BX 04,1125.115234,631.482986
2_BX 05,1258.221529,694.851794


In [48]:
df2018.to_csv('SNAP_2018.csv')
df2019.to_csv('SNAP_2019.csv')
df2020.to_csv('SNAP_2020.csv')

In [49]:
testdf=pd.read_csv('SNAP_2018.csv')
testdf

,index,yearly_ind_benefi_per_1000,yearly_household_benefi_per_1000
0,2_BX 01,975.070221,516.814759
1,2_BX 02,852.275772,459.614133
2,2_BX 03,1039.141446,554.060831
3,2_BX 04,786.692251,427.680772
4,2_BX 05,873.814353,469.820593
5,2_BX 06,935.833694,496.685401
6,2_BX 07,666.211967,369.943857
7,2_BX 08,320.207213,190.443424
8,2_BX 09,612.659462,331.948136
9,2_BX 10,291.115689,175.933617
